In [1]:
from os import path
from bs4 import BeautifulSoup
import pandas as pd

METADATA_DESTINATION = '../dataset/imdb_crop/'
METADATA_CROPPED_FILE_MATLAB = METADATA_DESTINATION + 'imdb.mat'

In [2]:
''' RETRIEVING NAMES
names = open('names.html', 'r', errors='ignore')
actors = open("500actors.txt", 'w')

actors_names = []
soup = BeautifulSoup(names, 'html.parser')
for s in soup.findAll('span'):
    if not('<img' in str(s.contents[0])) and not('<span' in str(s.contents[0])):
        if not(s.contents[0].isdigit()) and not('%' in str(s.contents[0])):
            if s.contents[0] != 'Global Sites' and s.contents[0] != 'Company':
                actors_names.append(s.contents[0])

for a in actors_names:
    actors.write(a + ';')
'''

' RETRIEVING NAMES\nnames = open(\'names.html\', \'r\', errors=\'ignore\')\nactors = open("500actors.txt", \'w\')\n\nactors_names = []\nsoup = BeautifulSoup(names, \'html.parser\')\nfor s in soup.findAll(\'span\'):\n    if not(\'<img\' in str(s.contents[0])) and not(\'<span\' in str(s.contents[0])):\n        if not(s.contents[0].isdigit()) and not(\'%\' in str(s.contents[0])):\n            if s.contents[0] != \'Global Sites\' and s.contents[0] != \'Company\':\n                actors_names.append(s.contents[0])\n\nfor a in actors_names:\n    actors.write(a + \';\')\n'

In [3]:
actors = open("500actors.txt", 'r')
actors = actors.read().split(';')[:-1]
len(actors)

1481

In [4]:
''' 500ACTORS IN IMDB
head, tail = path.split(METADATA_CROPPED_FILE_MATLAB)
path_serialized = path.join(head, 'imdb.pickle')
df = pd.read_pickle(path_serialized)
df.head()

df_500actors = pd.DataFrame(columns=df.columns)
for actor in actors:
    q = 'name=="' + actor + '"'
    df_500actors = pd.concat([df_500actors, df.query(q)])
df_500actors.reset_index()

len(df_500actors)

df_500actors = df_500actors[df_500actors.full_path != "04/nm0000204_rm3206975232_1981-6-9_2002.jpg"]
len(df_500actors)

head, tail = path.split(METADATA_CROPPED_FILE_MATLAB)
path_serialized = path.join(head, 'imdb_500actors.pickle')
df_500actors.to_pickle(path_serialized)

df_500actors = pd.read_pickle(path_serialized)
df_500actors.head()

df_500actors.describe()
'''

' 500ACTORS IN IMDB\nhead, tail = path.split(METADATA_CROPPED_FILE_MATLAB)\npath_serialized = path.join(head, \'imdb.pickle\')\ndf = pd.read_pickle(path_serialized)\ndf.head()\n\ndf_500actors = pd.DataFrame(columns=df.columns)\nfor actor in actors:\n    q = \'name=="\' + actor + \'"\'\n    df_500actors = pd.concat([df_500actors, df.query(q)])\ndf_500actors.reset_index()\n\nlen(df_500actors)\n\ndf_500actors = df_500actors[df_500actors.full_path != "04/nm0000204_rm3206975232_1981-6-9_2002.jpg"]\nlen(df_500actors)\n\nhead, tail = path.split(METADATA_CROPPED_FILE_MATLAB)\npath_serialized = path.join(head, \'imdb_500actors.pickle\')\ndf_500actors.to_pickle(path_serialized)\n\ndf_500actors = pd.read_pickle(path_serialized)\ndf_500actors.head()\n\ndf_500actors.describe()\n'

In [80]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
# From https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries/examples#Cats
sparql.setQuery("""
SELECT  DISTINCT ?gender (SAMPLE(?birth) as ?birth_date)
 (SAMPLE(?death_date) as ?dateOfDeath)  WHERE {
?id wdt:P31 wd:Q5.
?id wdt:P569 ?birth .
?id wdt:P21 ?gender.
OPTIONAL{?id wdt:P570 ?death_date .}
SERVICE wikibase:label {
bd:serviceParam wikibase:language "en".
?id rdfs:label ?idLabel .
}?id ?label "Natalie Portman"@en.
}
GROUP BY ?gender
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
results_df = pd.io.json.json_normalize(results['results']['bindings'])

  results_df = pd.io.json.json_normalize(results['results']['bindings'])



In [81]:
# male Q6581097
# female Q6581072
print(results_df['gender.value'].values[0])
print(results_df['birth_date.value'].values[0])

http://www.wikidata.org/entity/Q6581072
1981-06-09T00:00:00Z
